In [1]:
!pip install llama-index

  Obtaining dependency information for pydantic>=1.10 from https://files.pythonhosted.org/packages/e5/f3/8296f550276194a58c5500d55b19a27ae0a5a3a51ffef66710c58544b32d/pydantic-2.6.4-py3-none-any.whl.metadata
  Using cached pydantic-2.6.4-py3-none-any.whl.metadata (85 kB)
Using cached pydantic-2.6.4-py3-none-any.whl (394 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.6.4 which is incompatible.


In [2]:
%pip install llama-index-llms-together

Note: you may need to restart the kernel to use updated packages.


In [3]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
# from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

In [4]:
!pip install python-docx

In [12]:
wiki_titles=["output_text"]

In [6]:
from docx import Document

def extract_text_from_docx(docx_file_path):
    # Load the document
    doc = Document(docx_file_path)

    # Initialize an empty string to store the extracted text
    text = ''

    # Iterate over each paragraph in the document and extract text
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'

    return text

def save_text_to_txt(text, txt_file_path):
    # Save the extracted text to a .txt file
    with open(txt_file_path, 'w', encoding='utf-8') as file:
        file.write(text)

# Path to the document file
docx_file_path = 'New User Adoption Execution Steps.docx'

# Path to the .txt file where you want to store the extracted text
txt_file_path = 'output_text.txt'

# Extract text from the document
extracted_text = extract_text_from_docx(docx_file_path)

# Save the extracted text to a .txt file
save_text_to_txt(extracted_text, txt_file_path)

print("Text extracted from the document and saved to", txt_file_path)

Text extracted from the document and saved to output_text.txt


In [56]:
# Load  documents
# city_docs = []
# for wiki_title in wiki_titles:
#     docs = SimpleDirectoryReader(
#         input_files=[f"./{wiki_title}.docx"]
#     ).load_data()
#     docs[0].doc_id = wiki_title
#     city_docs.extend(docs)

In [15]:
splitter = SentenceSplitter(chunk_size=1000)

In [16]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [17]:
import nest_asyncio

nest_asyncio.apply()

In [18]:
from langchain_together import Together
splitter = SentenceSplitter(chunk_size=1000)
llm= Together(
        model="togethercomputer/llama-2-70b-chat",
        # model="gpt-4",
        temperature=0.1,
#         max_tokens=207,
        top_k=1,
        together_api_key="1cecf43792b1187b044ab0293853353cc45caf8fdfe0a82e049d53cbf5954d26",
    )

In [20]:
from llama_index.embeddings.together import TogetherEmbedding

embed_model = TogetherEmbedding(
    model_name="togethercomputer/m2-bert-80M-8k-retrieval", api_key="1cecf43792b1187b044ab0293853353cc45caf8fdfe0a82e049d53cbf5954d26")

In [21]:
from llama_index.core import Settings
Settings.context_window = 3000
Settings.llm =llm

In [22]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
# from sentence_transformers import SentenceTransformer, util
import os
from llama_index.core import Settings

Settings.llm =llm
# from llama_index.core import Settings
# Settings.context_window = 4096
# Settings.llm =llm
Settings.embed_model = embed_model

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

In [23]:
doc_summary_index = DocumentSummaryIndex.from_documents(
    city_docs,
    llm=llm,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: output_text


C:\Users\sumanyu.p\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
doc_summary_index.get_document_summary("output_text")

'\nThe provided text is about the user adoption execution steps for the sales Genie dashboard in the last one week. It includes the steps to refresh the dashboard, take backups for all three dashboards from Tableau, and download data in excel for all six regions. It also includes the steps to filter the data according to created at local date and time in SQL, import data into the test environment, and check for null Comex IT flags and get user details.\n\nThis text can answer questions such as:\n\n1. What are the steps involved in refreshing the sales Genie dashboard?\n2. How do you take backups for all three dashboards from Tableau?\n3. How do you download data in excel for all six regions?\n4. How do you filter data according to created at local date and time in SQL?\n5. How do you import data into the test environment?\n6. How do you check for null Comex IT flags and get user details?\n7. What are the regions included in the sales Genie dashboard?\n8. What is the purpose of the user

In [25]:
doc_summary_index.storage_context.persist("index")

In [26]:
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="index")
doc_summary_index = load_index_from_storage(storage_context)

In [28]:
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

In [29]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage,MessageRole

chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Summarize the documents.\n"
             "Always answer the query using the provided context information, "
            

        ),
    ), 
    ChatMessage(
        role=MessageRole.USER,
        content=(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information , "
    "answer the question: {query_str} in bullet points or numbered list where appropriate.\n"
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
query="""please summarize the documents"""

In [30]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query)

In [31]:
print(response__1)

 Sure, here's a summary of the document in bullet points:

• User adoption is used to check how many users have used the sales Genie Dashboard in the last one week.

• To refresh the dashboard, take backups of all three dashboards from Tableau and update the data in excel for all six regions.

• After downloading the data, open the excel sheet, delete the first column, and add two columns (Country & function) in each region.

• Filter the data according to created_at_local in SQL.

• Take a filtered data in excel and copy that data in a new excel sheet and save it.

• Open a remote desktop connection, go to SSMS, copy all excels, and paste them in the Bang Account remote desktop.

• Connect to the test environment, select the database (APACBI_FIGO_SALES), right-click, select Task, Import data, and choose a data source window.

• Select Data source as Micro soft excel, browse your excel sheet (single sheet at a time), click Next, and choose a Destination window.

• Select data base (SQL

In [32]:
query1="What are User Adoption Execution Steps? Summarize."

In [33]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query1)

In [34]:
print(response__1)

 Sure! Here are the User Adoption Execution Steps:

1. Take backups for all 3 dashboards from Tableau.
2. Connect to test database in SQL.
3. Take backups for all tables from the database.
4. Download data in excel for all 6 regions.
5. Open the excel sheet and delete the first column.
6. Add two columns (Country & function) in each region.
7. Remove 1 or 2 next to the QARA Dashboard in the NPI column.
8. Check for data created at local date and time.
9. Filter the data according to created_at_local in SQL.
10. Take a filtered data in excel and copy that data in a new excel sheet and save it.
11. Open the remote desktop connection, go to SSMS, copy all excels, and paste them in the Bang Account remote desktop.
12. Go to SSMS, connect to the Test Environment, select Database (APACBI_FIGO_SALES), right-click, select Task, import data.
13. Choose a data source window, select Data source as Micro soft excel, browse your excel sheet (single sheet at a time), click Next.
14. Choose a Destina

In [35]:
query2="Which link I have to use for taking backup of dashboard?"

In [36]:
response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query2)

In [37]:
print(response__2)



* Use the link <https://tableau-qa.medtronic.com/#/site/APACBI> to access the QA server.
* Click on the "Sales Genie" tab.
* Select the "Sales Genie user analytics" option for ANZ/SGMY/TH, "Sales Genie user analytics(JP_KR)" for KR/JP, and "NPI Genie User Anlaytics" for NPI.
* Click on the "Download" button to download the data in excel for all 6 regions.
* Use the link <https://tableau.medtronic.com/#/views/TableauUsageDetails_16226544435390/TraffictoViews?:iid=1> to download the data in excel for all 6 regions.

Please note that the provided links are for reference purposes only and may not be accessible outside of the Medtronic network.


In [38]:
query3=" Provide me details of “MDT Network Ids"

In [39]:
response__3 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query3)

In [40]:
print(response__3)

 Sure, here are the details of "MDT Network Ids" in bullet points:

* MDT Network Ids are used to identify and manage user adoption in the sales Genie Dashboard.
* The Ids are created and managed by the Medtronic IT team.
* There are four MDT Network Ids for different regions: ANZ/SGMY/TH, KR/JP, NPI, and Japan/Korea.
* Each Id has a unique name and description, such as "ANZ/SGMY/TH - Sales Genie User Analytics" and "KR/JP - Sales Genie User Analytics (JP_KR)".
* The Ids are used to filter and analyze user adoption data in the sales Genie Dashboard.
* The data is refreshed every Friday in QA, and the MDT Network Ids are used to identify and update the data for each region.
* The updated data is then loaded into the FIGO_SALES.ALL_SALES_GENIE_USERS table.
* The MDT Network Ids are also used to run stored procedures, such as dbo.APAC_Usage_Sales_Genie, to update the Comex IT flag for users.
* The flag is updated based on the user's country and region.
* The updated flag is then reflected

In [41]:
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
)

In [44]:
retriever = DocumentSummaryIndexLLMRetriever(
    doc_summary_index,
#     choice_select_prompt=None,
#     choice_batch_size=3,
    choice_top_k=1,
#     format_node_batch_fn=None,
#     parse_choice_select_answer_fn=None,
)

In [46]:
retrieved_nodes = retriever.retrieve(" How do you download data in excel for all six regions?")

In [47]:
print(len(retrieved_nodes))

2


In [48]:
print(retrieved_nodes[0].score)
print(retrieved_nodes[0].node.get_text())

10.0
User Adoption Execution Steps
User adoption is used to check how many users have used the sales Genie Dashboard in the last one week. This dashboard is updated every Friday in QA, below are the following steps to refresh this Dashboard: -
First take backups for all the 3 dashboards from Tableau 
Dashboards:
For taking backup of dashboard use below link:
https://tableau-qa.medtronic.com/#/site/APACBI
ANZ/SGMY/TH: Sales Genie-> Sales Genie user analytics
KR/JP: Sales Genie-> Sales Genie user analytics(JP_KR)
NPI: NPI_Genie-> NPI Genie User Anlaytics
We need to connect to test database in sql.
Take backups for all table from database
For ANZ/SGMY/TH use below two queries to take a backup 
Select * into ALL_SALES_GENIE_USERS_11_Jan_2024_BKP from [FIGO_SALES].[ALL_SALES_GENIE_USERS] ;  
Select * into dbo.APAC_USAGE_HIST_SNAPSHOT_ALL_QTR _11_Jan_2024_BKP  from dbo.APAC_USAGE_HIST_SNAPSHOT_ALL_QTR;
KR/JP use below two queries to take a backup
Select * into ALL_SALES_GENIE_USERS_11_Jan_20

In [49]:
# use retriever as part of a query engine
from llama_index.core.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [54]:
# query
response = query_engine.query("summarize the document?")
print(response)


The document describes the process for refreshing the user adoption dashboard in QA for various regions such as ANZ, SGMY, TH, KR, JP, and NPI. The steps include taking backups of dashboards from Tableau, connecting to test databases, taking backups of tables, downloading data in excel, filtering data based on created date and time, and importing data into SQL server. Additionally, it mentions checking for NULL Comex IT flags and updating user details in the FIGO_SALES.ALL_SALES_GENIE_USERS table. Finally, it lists the stored procedures to run and dashboards to refresh for each region.


In [53]:
response__11 = query_engine.query("what is effective tax rate?")
print(response__11)

Empty Response


In [ ]:
response__11 = query_engine.query("what is effective tax rate?")
print(response__11)